In [1]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("..") 
from gcforest.gcforest import GCForest

/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 数据及参数

In [2]:
random_seed = 42
cv=5
score = 'f1_weighted'

In [3]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = random_seed
    ca_config["max_layers"] = 10
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 6
    ca_config["estimators"] = []
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [4]:
path = os.getcwd()+'/../data/20122018freshwater_four_feature.csv'
data = pd.read_csv(path, na_values = np.nan)

In [5]:
X = data.drop(['本周水质'], axis=1).values # Series
y = data['本周水质'].values.reshape(-1,1) - 1

# 1. 中位数填充缺失值，2.Z-score标准化
clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
                           ('std_scaler', preprocessing.StandardScaler())])
X = clean_pipeline.fit_transform(X)

In [6]:
X.shape

(33612, 4)

# k折交叉验证

In [ ]:
# function：使用5折交叉验证统计各类别5次平均后的Acc，5次平均后的F1，和模型的总Acc以及总Weighted F1

# Input: 
#     X: 总样本
#     y: 总样本
#     model: function
#     cv: cross_validation的次数
# Output:
#     Acc_mean, 各类别的Acc
#     F1_mean, 各类别F1
#     Support_mean, 各类别预测样本占总样本的比重
#     Acc, 总Acc
#     F1_weighted 总Weighted F1


def kftrain(X, y, model, cv):
    model_name = model.__class__.__name__

    n_samples = X.shape[0]
    n_features = X.shape[1]
    n_class = np.unique(y).shape[0]
    Acc_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组test data的cv个acc
    F1_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组test data的cv个acc
    cv_F1_weighted = np.zeros(cv) # 5次的F1_weighted
    cv_Acc = np.zeros(cv) # 5次的Acc
    # 各类别的Support，每个类别的support由5次fold后得到的5个support求平均得到，这里support：预测为该类别的样本占总样本的比例
    # 各类别的F1，由5次fold后得到的5个F1求平均得到
    # 所以，Weighted F1 等于“5次fold得到的5个 Weighted F1 求平均得到” ——等价于—— “6个support*6个F1”得到
    Support_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 5次fold后各类别的support
    
    # 各类别的Mean，SD
    Acc_mean_class = np.zeros(6)
    Acc_SD_class = np.zeros(6)
    F1_mean_class = np.zeros(6)
    F1_SD_class = np.zeros(6)
    
    k = 0
    skf = StratifiedKFold(n_splits=cv) # 定义5折分层划分器
    
    for train_index, test_index in skf.split(X, y):
        K_train_x, K_test_x = X[train_index], X[test_index]
        K_train_y, K_test_y = y[train_index], y[test_index]
        
        if model_name == 'GCForest':
            model.fit_transform(K_train_x, K_train_y.reshape(K_train_y.shape[0]))
        else:
            model.fit(K_train_x, K_train_y)
    
        K_test_y_pred = model.predict(K_test_x)
        
        # 由混淆矩阵计算各类别的Acc
        test_cm = confusion_matrix(K_test_y, K_test_y_pred)
        test_acc_all_class = np.zeros(n_class) # 6个类别上的测试acc
        i = 0
        for c in test_cm:
            test_acc_all_class[i] = c[i]/np.sum(c)
            i += 1
        Acc_matrix[:,k] = test_acc_all_class
        
        # 由classification_report提取f1
        cr = classification_report(K_test_y, K_test_y_pred, digits=4)
        test_f1_all_class = np.zeros(n_class) # 6个类别上的测试f1
        support_all_class = np.zeros(n_class) # 6个类别的support
        i = 0
        for l in range(2,8):
            test_f1_all_class[i] = float(cr.splitlines()[l].split()[3])
            support_all_class[i] = float(cr.splitlines()[l].split()[4])/n_samples
            i = i + 1
        F1_matrix[:,k] = test_f1_all_class
        Support_matrix[:,k] = support_all_class
        
        # cv_F1_weighted[k] = float(cr.splitlines()[9].split()[5]) # 第k折时的Weighted F1
        cv_F1_weighted[k] = f1_score(K_test_y, K_test_y_pred, average="weighted")
        cv_Acc[k] = accuracy_score(K_test_y, K_test_y_pred)
        
        k += 1
    
    Acc_mean = np.mean(Acc_matrix, axis=1) # 6个类别的k-fold平均acc
    
    F1_mean = np.mean(F1_matrix, axis=1) # 6个类别的k-fold平均F1
    Support_mean = np.mean(Support_matrix, axis=1) # 6个类别的k-fold平均support
    
    Acc = np.mean(Acc_mean)
    F1_weighted = np.mean(cv_F1_weighted) # 5个Weighted F1 求平均，并可以证明其等价于 np.sum(F1_mean*Support_mean)
    SD_Acc = np.std(cv_Acc)
    SD_F1 = np.std(cv_F1_weighted)
    
    Acc_mean_class = np.mean(Acc_matrix, axis=1)
    Acc_SD_class = np.std(Acc_matrix, axis=1)
    F1_mean_class = np.mean(F1_matrix, axis=1)
    F1_SD_class = np.std(F1_matrix, axis=1)
    
    return Acc_matrix, F1_matrix, Support_matrix, Acc_mean, F1_mean, Support_mean, Acc, F1_weighted, SD_Acc, SD_F1, Acc_mean_class, Acc_SD_class, F1_mean_class, F1_SD_class

In [ ]:
config = get_toy_config()

models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    SVC(probability=True),
    DecisionTreeClassifier(),
    ExtraTreeClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=random_seed),
    ExtraTreesClassifier(random_state=random_seed),
    GCForest(config)
]

for model in models:
    model_name = model.__class__.__name__
    print(model_name)
    Acc_matrix, F1_matrix, Support_matrix, Acc_class, F1_class, Support_class, Acc, F1_weighted, SD_Acc, SD_F1, Acc_mean_class, Acc_SD_class, F1_mean_class, F1_SD_class = kftrain(X, y, model, 5)
    print("===Accuracy===")
    for i in range(len(Acc_class)):
        print("Class %d: %.2f%%" %(i, Acc_class[i]*100))
        print("Mean±SD: %.2f±%.2f%%" %(Acc_mean_class[i]*100,Acc_SD_class[i]*100))
        
    print("Overall: %.2f%%" %(Acc*100))
    print("Mean±SD: %.2f±%.2f%%" %(Acc*100,SD_Acc*100))

    print("===   F1  ===")
    for i in range(len(F1_class)):
        print("Class %d: %.2f%%" %(i, F1_class[i]*100))
        print("Mean±SD: %.2f±%.2f%%" %(F1_mean_class[i]*100,F1_SD_class[i]*100))
        
    print("F{beta}: %.2f%%" %(F1_weighted*100))
    print("Mean±SD: %.2f±%.2f%%" %(F1_weighted*100,SD_F1*100))
    print("\n===============\n")
    
    print("Acc_matrix")
    print(Acc_matrix)
    print("F1_matrix")
    print(F1_matrix)
    print("Support_matrix")
    print(Support_matrix)

LogisticRegression


/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


===Accuracy===
Class 0: 5.74%
Mean±SD: 5.74±3.20%
Class 1: 99.17%
Mean±SD: 99.17±0.33%
Class 2: 57.64%
Mean±SD: 57.64±2.91%
Class 3: 44.13%
Mean±SD: 44.13±3.79%
Class 4: 0.00%
Mean±SD: 0.00±0.00%
Class 5: 75.87%
Mean±SD: 75.87±7.48%
Overall: 47.09%
Mean±SD: 47.09±0.94%
===   F1  ===
Class 0: 10.60%
Mean±SD: 10.60±5.74%
Class 1: 80.26%
Mean±SD: 80.26±0.50%
Class 2: 60.59%
Mean±SD: 60.59±1.72%
Class 3: 51.43%
Mean±SD: 51.43±3.03%
Class 4: 0.00%
Mean±SD: 0.00±0.00%
Class 5: 76.78%
Mean±SD: 76.78±5.36%
F{beta}: 61.59%
Mean±SD: 61.59±0.78%


Acc_matrix
[[0.08811475 0.00819672 0.0307377  0.08213552 0.07802875]
 [0.98568738 0.99246704 0.99171063 0.99321778 0.99547852]
 [0.59147727 0.60795455 0.5997726  0.53780557 0.54519613]
 [0.38082192 0.49680365 0.42595978 0.45155393 0.45155393]
 [0.         0.         0.         0.         0.        ]
 [0.68372093 0.75058275 0.85314685 0.83449883 0.67132867]]
F1_matrix
[[0.1593 0.0163 0.0596 0.1501 0.1445]
 [0.8019 0.8082 0.8086 0.7976 0.7969]
 [0.6005 0.

/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/ipykernel/__main__.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


===Accuracy===
Class 0: 76.70%
Mean±SD: 76.70±4.29%
Class 1: 93.59%
Mean±SD: 93.59±0.77%
Class 2: 89.85%
Mean±SD: 89.85±1.09%
Class 3: 89.77%
Mean±SD: 89.77±1.76%
Class 4: 78.95%
Mean±SD: 78.95±2.81%
Class 5: 88.63%
Mean±SD: 88.63±2.22%
Overall: 86.25%
Mean±SD: 86.25±0.74%
===   F1  ===
Class 0: 77.11%
Mean±SD: 77.11±2.97%
Class 1: 92.00%
Mean±SD: 92.00±0.70%
Class 2: 90.32%
Mean±SD: 90.32±0.70%
Class 3: 90.86%
Mean±SD: 90.86±1.01%
Class 4: 79.92%
Mean±SD: 79.92±2.21%
Class 5: 92.36%
Mean±SD: 92.36±1.42%
F{beta}: 89.78%
Mean±SD: 89.78±0.76%


Acc_matrix
[[0.73155738 0.81967213 0.80532787 0.77207392 0.7063655 ]
 [0.93483992 0.93446328 0.93406179 0.92614921 0.94988696]
 [0.88636364 0.9125     0.89653212 0.88743604 0.90960773]
 [0.88675799 0.90410959 0.92321755 0.90310786 0.87111517]
 [0.73489933 0.81208054 0.8047138  0.79124579 0.8047138 ]
 [0.88139535 0.8974359  0.8951049  0.91142191 0.84615385]]
F1_matrix
[[0.7604 0.8155 0.7955 0.7373 0.747 ]
 [0.9128 0.9299 0.9216 0.9114 0.9241]
 [0.8

...

[ 2019-05-11 14:48:45,375][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_3 - 5_folds.train_3.predict)=99.50%
[ 2019-05-11 14:48:45,597][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_3 - 5_folds.train_4.predict)=99.55%
[ 2019-05-11 14:48:45,603][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_3 - 5_folds.train_cv.predict)=99.55%
[ 2019-05-11 14:48:45,810][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_4 - 5_folds.train_0.predict)=98.83%
[ 2019-05-11 14:48:46,038][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_4 - 5_folds.train_1.predict)=99.16%
[ 2019-05-11 14:48:46,263][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_4 - 5_folds.train_2.predict)=99.14%
[ 2019-05-11 14:48:46,474][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_4 - 5_folds.train_3.predict)=99.26%
[ 2019-05-11 14:48:46,692][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_4 - 5_folds

[ 2019-05-11 14:48:56,484][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_1.predict)=98.87%
[ 2019-05-11 14:48:56,531][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_2.predict)=99.00%
[ 2019-05-11 14:48:56,577][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_3.predict)=98.96%
[ 2019-05-11 14:48:56,620][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_4.predict)=98.77%
[ 2019-05-11 14:48:56,626][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_cv.predict)=98.91%
[ 2019-05-11 14:48:56,682][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_5 - 5_folds.train_0.predict)=99.03%
[ 2019-05-11 14:48:56,724][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_5 - 5_folds.train_1.predict)=98.98%
[ 2019-05-11 14:48:56,766][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_5 - 5_folds

[ 2019-05-11 14:49:06,828][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_1.predict)=98.81%
[ 2019-05-11 14:49:07,101][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_2.predict)=98.68%
[ 2019-05-11 14:49:07,404][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_3.predict)=98.68%
[ 2019-05-11 14:49:07,674][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_4.predict)=99.05%
[ 2019-05-11 14:49:07,681][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_cv.predict)=98.86%
[ 2019-05-11 14:49:08,696][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_7 - 5_folds.train_0.predict)=99.61%
[ 2019-05-11 14:49:09,349][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_7 - 5_folds.train_1.predict)=99.41%
[ 2019-05-11 14:49:10,020][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_7 - 5_folds

[ 2019-05-11 14:49:23,729][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_1.predict)=99.50%
[ 2019-05-11 14:49:23,958][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_2.predict)=99.50%
[ 2019-05-11 14:49:24,182][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_3.predict)=99.57%
[ 2019-05-11 14:49:24,499][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_4.predict)=99.31%
[ 2019-05-11 14:49:24,506][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_cv.predict)=99.48%
[ 2019-05-11 14:49:24,749][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_1 - 5_folds.train_0.predict)=99.70%
[ 2019-05-11 14:49:25,010][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_1 - 5_folds.train_1.predict)=99.48%
[ 2019-05-11 14:49:25,269][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_1 - 5_folds

[ 2019-05-11 14:49:40,478][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_3.predict)=99.48%
[ 2019-05-11 14:49:40,715][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_4.predict)=99.55%
[ 2019-05-11 14:49:40,723][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_cv.predict)=99.47%
[ 2019-05-11 14:49:40,990][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds.train_0.predict)=99.35%
[ 2019-05-11 14:49:41,228][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds.train_1.predict)=99.53%
[ 2019-05-11 14:49:41,485][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds.train_2.predict)=99.40%
[ 2019-05-11 14:49:41,984][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds.train_3.predict)=99.44%
[ 2019-05-11 14:49:42,513][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds